In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 서울시 월별 화재발생 건수 데이터 (2021년/2022년)

In [ ]:
import pandas as pd

DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/멀티캠퍼스34/semi_project/data/"
df = pd.read_csv(DATA_PATH + "화재발생_자치구별_현황(월별).csv", encoding='CP949')
df.head()

,자치구,2021. 01,2021. 02,2021. 03,2021. 04,2021. 05,2021. 06,2021. 07,2021. 08,2021. 09,...,2022. 03,2022. 04,2022. 05,2022. 06,2022. 07,2022. 08,2022. 09,2022. 10,2022. 11,2022. 12
0,종로구,18,9,18,14,16,15,26,13,15,...,17,15,20,10,17,18,19,13,17,16
1,중구,19,7,16,11,7,11,19,19,20,...,16,15,12,15,17,16,10,13,16,15
2,용산구,14,18,14,22,15,14,13,9,15,...,21,21,20,13,16,20,10,14,18,25
3,성동구,17,17,15,14,15,19,17,18,13,...,17,19,28,13,19,16,16,17,9,23
4,광진구,17,12,13,16,20,19,26,11,15,...,9,13,15,12,15,12,8,15,8,12


In [ ]:
# Split the dataframe into 2021 and 2022 data
df_2021 = df.loc[:, ['자치구'] + [col for col in df.columns if '2021' in col]]
df_2022 = df.loc[:, ['자치구'] + [col for col in df.columns if '2022' in col]]

df_2021_melted = df_2021.melt(id_vars=['자치구'], var_name='월', value_name='화재발생건수')
df_2022_melted = df_2022.melt(id_vars=['자치구'], var_name='월', value_name='화재발생건수')

df_2021_melted['월'] = df_2021_melted['월'].apply(lambda x: x.split('.')[1].strip())
df_2022_melted['월'] = df_2022_melted['월'].apply(lambda x: x.split('.')[1].strip())

df_2021_melted['월'] = df_2021_melted['월'].astype(int)
df_2022_melted['월'] = df_2022_melted['월'].astype(int)

df_2021_melted = df_2021_melted.sort_values(['월', '화재발생건수'])
df_2022_melted = df_2022_melted.sort_values(['월', '화재발생건수'])

df_2022_melted.head()

,자치구,월,화재발생건수
14,양천구,1,7
19,동작구,1,11
6,중랑구,1,12
17,금천구,1,12
5,동대문구,1,13


In [ ]:
# 자치구별 막대그래프

import plotly.graph_objects as go
from plotly.subplots import make_subplots

for district in df_2021_melted['자치구'].unique():
    data_2021 = df_2021_melted[df_2021_melted['자치구'] == district]['화재발생건수']
    data_2022 = df_2022_melted[df_2022_melted['자치구'] == district]['화재발생건수']

    months = [f'{i}월' for i in range(1, 13)]

    fig = make_subplots(rows=1, cols=1)

    fig.add_trace(go.Bar(x=months, y=data_2021.values.flatten(), name='2021년',marker_color='lightsalmon'), row=1, col=1)
    fig.add_trace(go.Bar(x=months, y=data_2022.values.flatten(), name='2022년',marker_color='tomato'), row=1, col=1)

    fig.update_layout(title_text=f'{district} 월별 화재건수', barmode='group', template='plotly_white')
    fig.show()

In [ ]:
import plotly.express as px

def create_plotly_line_chart(df, year):
    fig = px.line(
        df,
        x='월',
        y='화재발생건수',
        color='자치구',
        markers=True,
        line_shape='spline',
        title=f'{year}년 월별 화재건수',
    )
    return fig

fig_2021 = create_plotly_line_chart(df_2021_melted, '2021')
fig_2021.show()

fig_2022 = create_plotly_line_chart(df_2022_melted, '2022')
fig_2022.show()

In [ ]:
# 연도별 자치구 화재건수 그래프

# import plotly.express as px

# def create_plotly_bar_chart(df, year):
#     fig = px.bar(
#         df,
#         x='월',
#         y='화재발생건수',
#         color='자치구',
#         title=f'{year}년 월별 화재건수',
#     )
#     fig.update_layout(barmode='group')
#     return fig

# fig_2021 = create_plotly_bar_chart(df_2021_melted, '2021')
# fig_2021.show()

# fig_2022 = create_plotly_bar_chart(df_2022_melted, '2022')
# fig_2022.show()

In [ ]:
# 상반기/하반기 그래프
# 오름차순 정렬 안됨,,,,,

def create_plotly_bar_chart2(df, year):
    fig = px.bar(
        df,
        x='월',
        y='화재발생건수',
        color='자치구',
        title=f'{year}'
    )
    fig.update_layout(barmode='group')
    return fig

def split_and_sort_data(df):
    first_half = df[df['월'].between(1, 6)]
    second_half = df[df['월'].between(7, 12)]

    return first_half, second_half

def create_and_show_charts(df, year):
    first_half, second_half = split_and_sort_data(df)

    fig_first_half = create_plotly_bar_chart2(first_half, f'{year}년 상반기 월별 화재건수')
    fig_second_half = create_plotly_bar_chart2(second_half, f'{year}년 하반기 월별 화재건수')

    fig_first_half.show()
    fig_second_half.show()

create_and_show_charts(df_2021_melted, '2021')
create_and_show_charts(df_2022_melted, '2022')

# 서울시 자치구/동별 화재발생 장소 데이터 (2021년/2022년)

In [ ]:
import pandas as pd

DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/멀티캠퍼스34/semi_project/data/"
df2021 = pd.read_csv(DATA_PATH + "화재발생_동별_2021.csv", encoding='CP949')
df2022 = pd.read_csv(DATA_PATH + "화재발생_동별_2022.csv", encoding='CP949')
df2021.head()

,자치구,동,단독주택,공동주택,기타주택,학교,일반업무,판매시설,숙박시설,종교시설,의료시설,공장 및 창고,작업장,위락오락시설,음식점,일상서비스시설,기타
0,종로구,사직동,-,1,-,1,1,1,-,-,-,-,-,-,3,-,1
1,종로구,삼청동,1,-,-,-,1,1,-,-,-,-,-,-,1,-,1
2,종로구,부암동,-,2,-,1,-,-,-,-,-,-,-,-,-,-,-
3,종로구,평창동,4,2,-,-,-,1,-,1,-,-,-,-,-,-,3
4,종로구,무악동,-,2,-,-,-,-,-,-,-,-,-,-,-,-,-


In [ ]:
def preprocess_data(df, start_column=2):
    df = df.replace('-', 0)

    columns = df.columns[start_column:]
    for column in columns:
        df[column] = df[column].astype(int)

    return df

df21 = preprocess_data(df2021)
df22 = preprocess_data(df2022)
print(df22.dtypes)

자치구        object
동          object
단독주택        int64
공동주택        int64
기타주택        int64
학교          int64
일반업무        int64
판매시설        int64
숙박시설        int64
종교시설        int64
의료시설        int64
공장 및 창고     int64
작업장         int64
위락오락시설      int64
음식점         int64
일상서비스시설     int64
기타          int64
dtype: object


In [ ]:
df22.head()

,자치구,동,단독주택,공동주택,기타주택,학교,일반업무,판매시설,숙박시설,종교시설,의료시설,공장 및 창고,작업장,위락오락시설,음식점,일상서비스시설,기타
0,종로구,사직동,1,0,0,0,0,1,0,0,0,0,0,0,3,0,3
1,종로구,삼청동,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1
2,종로구,부암동,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,종로구,평창동,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,종로구,무악동,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 파이 차트

import plotly.graph_objects as go
from plotly.subplots import make_subplots

def create_double_pie_chart(df1, title1, df2, title2):
    type_sums1 = df1.iloc[:, 2:].sum().sort_values(ascending=False)
    type_sums2 = df2.iloc[:, 2:].sum().sort_values(ascending=False)

    fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                        subplot_titles=[title1, title2])

    fig.add_trace(go.Pie(labels=type_sums1.index, values=type_sums1.values), 1, 1)
    fig.add_trace(go.Pie(labels=type_sums2.index, values=type_sums2.values), 1, 2)

    fig.update_traces(hole=.3)

    fig.update_layout(title_text='서울시 장소 유형별 화재 발생 비율')

    fig.show()

create_double_pie_chart(df21, '2021년', df22, '2022년')

In [ ]:
# 자치구별 트리맵 차트

import plotly.express as px
import pandas as pd

def create_treemap_charts(df1, title1, df2, title2, width, height):
    def melt_df(df):
        df_melted = df.drop('동', axis=1).melt(id_vars=['자치구'], var_name='항목', value_name='값')
        df_melted['값'] = pd.to_numeric(df_melted['값'], errors='coerce').fillna(0)
        return df_melted

    long_df1 = melt_df(df1)
    long_df2 = melt_df(df2)

    fig1 = px.treemap(long_df1, path=['자치구', '항목'], values='값', title=title1)
    fig1.update_layout(width=width, height=height)
    fig1.show()

    fig2 = px.treemap(long_df2, path=['자치구', '항목'], values='값', title=title2)
    fig2.update_layout(width=width, height=height)
    fig2.show()

create_treemap_charts(df21, '2021년 구별 화재발생 장소', df22, '2022년 구별 화재발생 장소', 1000, 1200)

In [ ]:
# 자치구/동별 화재장소 트리맵 차트

import plotly.express as px
import pandas as pd

def create_treemap_charts(df1, title1, df2, title2, width, height):
    def melt_df(df):
        return df.melt(id_vars=['자치구', '동'], var_name='항목', value_name='값')

    long_df1 = melt_df(df1)
    long_df2 = melt_df(df2)

    fig1 = px.treemap(long_df1, path=['자치구', '동', '항목'], values='값', title=title1)
    fig1.show()

    fig2 = px.treemap(long_df2, path=['자치구', '동', '항목'], values='값', title=title2)
    fig2.show()

create_treemap_charts(df21, '2021년 서울시 화재발생 장소', df22, '2022년 서울시 화재발생 장소', 1000, 1200)

In [ ]:
# 테이블 차트

import plotly.graph_objects as go

def cell_color(column, value):
    if column == '동':
        return 'lightpink'
    else:
        return 'orange' if value != 0 else 'ivory'

unique_districts = df21['자치구'].unique()

for district in unique_districts:
    district_df = df21[df21['자치구'] == district]
    columns = district_df.columns[1:]

    cell_colors = [[cell_color(col, district_df.iloc[i][col]) for i in range(len(district_df))] for col in columns]

    fig = go.Figure(data=[go.Table(
        header=dict(
            values=columns,
            fill_color='lightcoral',
            align='center'
        ),
        cells=dict(
            values=[district_df[col].tolist() for col in columns],
            fill_color=cell_colors,
            align='center'
        )
    )])

    fig.update_layout(title=f'2021년 {district} 화재장소별 화재발생 건수')
    fig.show()

In [ ]:
def cell_color(column, value):
    if column == '동':
        return 'lightpink'
    else:
        return 'orange' if value != 0 else 'ivory'

unique_districts = df22['자치구'].unique()

for district in unique_districts:
    district_df = df22[df22['자치구'] == district]
    columns = district_df.columns[1:]

    cell_colors = [[cell_color(col, district_df.iloc[i][col]) for i in range(len(district_df))] for col in columns]

    fig = go.Figure(data=[go.Table(
        header=dict(
            values=columns,
            fill_color='lightcoral',
            align='center'
        ),
        cells=dict(
            values=[district_df[col].tolist() for col in columns],
            fill_color=cell_colors,
            align='center'
        )
    )])

    fig.update_layout(title=f'2022년 {district} 화재장소별 화재발생 건수')
    fig.show()